In [ ]:
%pip install pandas
%pip install tqdm
%pip install psycopg2-binary

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'  # Sync errors
os.environ['TORCH_USE_CUDA_DSA'] = '1'    # Detailed asserts

In [ ]:
import pandas as pd
from tqdm import tqdm
from uuid import uuid5, UUID
import psycopg2

In [ ]:
%pip install dotenv

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
%pip install openai

In [ ]:
UUID_NAMESPACE = UUID("c87c53d6-4464-4018-b4c9-15718d354ec8")
UUID_NAMESPACE

# ARIA

In [ ]:
df = pd.read_csv("./accidents-tous-req10905.csv", encoding="cp1252", sep=";", skiprows=7)
print(df.info())

In [ ]:
print(df.head(1))

In [ ]:
def convert_to_db(df : pd.DataFrame, trunc = None):    

    if trunc is not None :
        df = df.head(trunc)

    def create_line(line : pd.Series):
        address = " ".join([str(line["Départment"]), str(line["Commune"])])
        site_id = str(uuid5(UUID_NAMESPACE, address))
        sites = {
            "site_id" : site_id,
            "plant_name": "",
            "address": address,
            "latitude": None,               # to fill later
            "longitude": None,              # to fill later
            "country": line["Pays"],
            "industrial_activity": line["Code NAF"],
        }

        accident_key = " ".join([str(line["Titre"]), str(line["Date"])])
        accident_id = str(uuid5(UUID_NAMESPACE, accident_key))
        accidents = {
            "accident_id": accident_id,
            "site_id": site_id,
            "title": line["Titre"],
            "source": "ARIA",
            "source_id": str(line["Numéro ARIA"]),
            "accident_date": line["Date"],
            "severity_scale": line["Echelle"],
            "raw_data": "", #line,
            "created_at": "date.now()",
            "updated_at": "",
        }

        causes = {
            "accident_id": accident_id,
            "event_category": line["Causes profondes"],
            "failure": line["Causes premières"],
            "description": line["Contenu"], # could also reuse Contenu
        }

        substances = {
            "accident_id": accident_id,
            "name": line["Matières"],
            "cas_number": "",
            "quantity": "",
            "clp_class" : line["Classe de danger CLP"]
        }

        consequences_human = {
            "accident_id": accident_id,
            "fatalities": None,
            "injuries": None,
            "evacuated": None,
            "hospitalized": None,
        }

        consequences = {
            "ENVIRONNEMENTALES" : "",
            "ÉCONOMIQUES" : ""
        }

        try :
            for consequence in line["Conséquences"].split("CONSÉQUENCES "):
                if len(consequence) < 2 : continue
                s = consequence.split(',')
                key = s[0]
                content = (','.join(s[1:])).removesuffix(',')
                consequences[key] = content
        except : pass

        consequences_other = {
            "accident_id": accident_id,
            "environmental_impact": consequences["ENVIRONNEMENTALES"],
            "economic_cost": consequences["ÉCONOMIQUES"],
            "disruption_duration": line["Type évènement"]
        }

        tables = {
            "sites": sites,
            "accidents": accidents,
            "causes": causes,
            "substances": substances,
            "consequences_human": consequences_human,
            "consequences_other": consequences_other
        }
        
        return tables
    
    db_lines = []

    for x in tqdm(iter(df.iloc), total=trunc, ncols=200):
        db_lines.append(create_line(x))

    return db_lines

ARIA_db_jsons = convert_to_db(df, trunc=1000)

In [ ]:
from IPython.display import JSON, display

def print_db_jsons(json):
    for i, db_line in enumerate(json):
        print(i, "=" * 200)
        for key in db_line :
            print(key, flush=True, end='')
            display(JSON(db_line[key], expanded=True))
        if i == 0 : break

print_db_jsons(ARIA_db_jsons)

# OSHA - Injuries (ITA)

In [ ]:
df = pd.read_csv("./ITA Case Detail Data 2024 through 08-31-2025.csv", sep=",")
print(df.info())

In [ ]:
print(df.head(1))

In [ ]:
def convert_to_db(df : pd.DataFrame, trunc = None):    

    if trunc is not None :
        df = df.head(trunc)

    def create_line(line : pd.Series):
        address = f"{line['street_address']} {line['city']} {line['state']} {line['zip_code']}"
        site_key = line["establishment_name"] + " " + address
        site_id = str(uuid5(UUID_NAMESPACE, site_key))
        sites = {
            "site_id" : site_id,
            "plant_name": line["establishment_name"],  # or fallback to "company_name"
            "address": address,
            "latitude": None,  # Geocode later from address
            "longitude": None,
            "country": "USA",
            "industrial_activity": str(line["naics_code"]),  # Maps to NAF equivalent
        }

        accident_key = " ".join([line["job_description"], line["date_of_incident"]])
        accident_id = str(uuid5(UUID_NAMESPACE, accident_key))
        accidents = {
            "accident_id": accident_id,
            "site_id": site_id,
            "title": line["job_description"],  # Brief incident summary
            "source": "OSHA ITA",
            "source_id": line["case_number"],  # Unique OSHA case identifier
            "accident_date": line["date_of_incident"],
            "severity_scale": int(line["incident_outcome"]),  # 1 = Death / 2 = Days away from work / 3 = Job transfer or restriction / 4 = Other recordable case
            "raw_data": "",# line.to_dict(),  # Full line as JSON
            "created_at": "date.now()",
            "updated_at": "",  # Fill on save
        }

        causes = {
            "accident_id": accident_id, 
            "event_category": line["NEW_NAR_WHAT_HAPPENED"],  # Deep/root causes
            "equipment_failure": line["NEW_NAR_BEFORE_INCIDENT"],  # Initial triggers
            "description": line["NEW_INCIDENT_DESCRIPTION"],
        }

        substances = {
            "accident_id": accident_id,
            "name": line["NEW_NAR_OBJECT_SUBSTANCE"],  # Object/substance hit/contacted
            "cas_number": "",  # Not in ITA; research via name if needed
            "quantity": "",  # Derive from context if available
            "clp_class": line["NEW_NAR_INJURY_ILLNESS"],  # Injury type as hazard proxy
        }

        consequences_human = {
            "accident_id": accident_id,
            "fatalities": 1 if pd.notna(line["date_of_death"]) else 0,
            "injuries": 1,  # Each line is one recordable case
            "evacuated": None,  # Not directly available
            "hospitalized": 1 if line["dafw_num_away"] > 0 else 0,  # Days away implies severity
        }

        consequences_other = {
            "accident_id": accident_id,
            "environmental_impact": "",  # ITA focuses on worker injuries
            "economic_cost": "",  # Estimate from total_hours_worked if needed
            "disruption_duration": int(line["djtr_num_tr"]),  # Restriction days as proxy
        }

        tables = {
            "sites": sites,
            "accidents": accidents,
            "causes": causes,
            "substances": substances,
            "consequences_human": consequences_human,
            "consequences_other": consequences_other
        }

        # taken_cols = ["Titre", "Pays", "Code NAF", "Numéro ARIA", "Date", "Echelle", "Causes profondes", "Causes premières", "Contenu", "Matières", "Conséquences", "Départment", "Commune", "Classe de danger CLP", "Type évènement"]
        # print(line.drop(labels=taken_cols, errors='ignore'))
        
        return tables
    
    db_lines = []

    if trunc is None:
        itr = iter(df.iloc)
    else :
        itr = iter(df.head(trunc).iloc)

    for x in tqdm(itr, total=5, ncols=200):
        db_lines.append(create_line(x))

    return db_lines

# OSHA_db_jsons = convert_to_db(df, trunc=500)

In [ ]:
# print_db_jsons(OSHA_db_jsons)

# EPICEA

In [ ]:
epicea_example = {
    "Numéro du dossier": "27615",
    "Comité technique national": "B - Bâtiment et Travaux Publics",
    "Code entreprise": "4321A - Travaux d'installation électrique dans tous locaux",
    "Matériel en cause": "270302 - Chaudière à mazout",
    "Résumé de l'accident": r"Une équipe de trois salariés dont un plombier chauffagiste, âgé de 30 ans, intervient chez un particulier, un vétérinaire dont l'habitation se compose d'une maison et d'un bloc opératoire (achetée il y a 21 ans auparavant, maison construite dans les années 70). L'installation de chauffage de l'habitation, composée d’une pompe à chaleur et d’une chaudière au fioul, alimente les 28 radiateurs de la propriété. L'alimentation en eau chaude est assurée par un ballon d'eau indépendant de l'installation de chauffage. Une chaudière au fioul de la marque *** est placée dans un local à proximité de l'habitation. La chaudière fonctionne en relai de la pompe à chaleur. Quand la pompe n'arrive plus à maintenir la consigne de température, la chaudière au fioul fonctionne (le propriétaire de l’installation ne dispose pas du manuel d'utilisation ou d'installation de la chaudière). Le propriétaire rencontre des difficultés sur le fonctionnement de la chaudière au fioul mais ne valide pas le premier devis pour la remplacer. Le second devis propose une remise en état de la chaudière existante avec un désembouage des radiateurs, le remplacement des vannes des radiateurs, l'alimentation entre la cuve de fioul et la chaudière et le ramonage. Ce devis est validé par le propriétaire. Le jour de l’accident, l’équipe intervient pour cette prestation approuvée. Le plombier chauffagiste réalise seul l'entretien de la chaudière dans le local dédié et ses deux collègues réalisent le désembouage des radiateurs dans l’habitation. Pour réaliser le test de combustion, le plombier chauffagiste allume la chaudière. Il met la sonde de l'analyseur de combustion au niveau de l'évacuation des fumées. Suivant le brûleur et les valeurs fournies par l'analyseur, il règle la chaudière. Il est accroupi à proximité de la chaudière. La chaudière explose, le local de la chaudière est envahi de fumée et de vapeurs d'eau. A la suite de l’explosion, le brûleur de la chaudière est au sol devant la chaudière. Il a été éjecté. La trappe d'accès de la chambre de combustion, au-dessus du brûleur, est cassée sur un angle. A l'intérieur de la chambre de combustion, dans le fond, la cuve en fonte présente un trou. Le plombier chauffagiste se dirige vers la sortie, puis vers le véhicule de l'entreprise. Il se regarde dans le rétroviseur du véhicule car son visage le brûle. Il constate qu'il n'est pas brûlé sur le visage. Il se déshabille lui-même. Son collègue appelle les pompiers. A leur arrivée, le plombier est dans la baignoire de l'habitation à une eau tempérée, toujours conscient. Il est brûlé par de la vapeur d’eau sur plus de 50 % du corps aux niveaux des bras, du torse, des parties intimes et des jambes. Hypothèses pour expliquer l’explosion de la chaudière : sur le départ du réseau d'eau de chauffage de la chaudière, il y a des dispositifs de sécurité : un vase d'expansion suivi d'une soupape de sécurité. Le vase d'expansion compense le surplus de volume d'eau quand elle chauffe, il se déclenche quand la température atteint 60°C. La soupape de sécurité se déclenche si la pression atteint les 3 bars. La vanne purge l'eau du circuit pour qu'elle s'écoule dans le local de la chaudière, hors du réseau d'eau de chauffage. Le jour de l'accident, la vanne d’isolement (entre la chaudière et le réseau d’eau de l’habitation pour le chauffage) est fermée. Ce qui permet au plombier chauffagiste de réaliser le contrôle de combustion sur la chaudière et à ses collègues de continuer le désembouage sur les radiateurs. Cette vanne est située avant les dispositifs de sécurité de la chaudière. De ce fait, lorsque le plombier chauffagiste a allumé la chaudière, la température de l'eau augmente ce qui engendre une élévation de la pression. Les dispositifs de sécurité étant situés après la vanne d'isolement, ils ne se déclenchent pas lorsque la température et la pression atteignent les valeurs de déclenchement. La chaudière continue de chauffer l’eau jusqu’à ce qu’elle se transforme en vapeur. La pression dans le circuit de la chaudière est telle que la chaudière explose avec la libération de la vapeur d'eau dans le local."
}

len(epicea_example["Résumé de l'accident"].split(' '))

In [ ]:
import hashlib
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from typing import Dict, Any
import json

In [ ]:
from typing import List
from pydantic import Field


OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(
    model="gpt-5-nano",
    stream_usage=True,
    reasoning_effort="low",
    service_tier="flex"
)

SYSTEM_MESSAGE = SystemMessage(content="Tu es un assistant français. Réponds UNIQUEMENT en français avec des réponses courtes et précises. Ne jamais utiliser l'anglais." \
            "Tu es un expert en extraction de données. Il va t'etre passé du text non structuré, et tu dois le convertir dans la structure donnée. " \
            "Si un nombre n'est pas mentionné, répond 0")

class NumberSchema(BaseModel):
    response:int

class TextSchema(BaseModel):
    response:str

class Substance(BaseModel):
    """Single chemical substance involved in incident."""
    name: str = Field(..., description="Chemical name")
    cas_number: str = Field(..., description="CAS registry number")
    quantity: str = Field(..., description="Quantity released/spilled")
    clp_class: str = Field(..., description="CLP hazard classification")

class SubstancesOutput(BaseModel):
    """Extracted substances from accident report (0 to many)."""
    response: List[Substance] = Field(default_factory=list, description="List of substances")

class DiskCache:
    def __init__(self, cache_file: str = "cache.json"):
        self.cache_file = cache_file
        try:
            with open(cache_file, 'r') as f:
                self.cache: Dict[str, Dict[str, Any]] = json.load(f)
        except FileNotFoundError:
            self.cache = {}
    
    def get(self, context_hash: str, schema):
        if context_hash in self.cache:
            data = self.cache[context_hash]
            return schema(**data)
        return None
    
    def set(self, context_hash: str, result):
        self.cache[context_hash] = result.model_dump()
        with open(self.cache_file, 'w') as f:
            json.dump(self.cache, f)
    
    def extract(self, context: str, schema, force_run : bool = False):
        context_hash = hashlib.md5(context.encode()).hexdigest()
        
        if not force_run :
            cached = self.get(context_hash, schema)
            if cached:
                print(f"✅ Cache HIT for context hash: {context_hash[:8]}")
                return cached
        
        print(f"🔄 Cache MISS - calling API for hash: {context_hash[:8]}")
        
        messages = [
            SYSTEM_MESSAGE,
            HumanMessage(content=context)
        ]

        structured_llm = llm.with_structured_output(schema, include_raw=True)
        result = structured_llm.invoke(messages)
        
        self.set(context_hash, result["parsed"])
        return result["parsed"]

context = epicea_example["Résumé de l'accident"]
cache = DiskCache()
result = cache.extract(f"Extrait le nombre d'accidentés de ceci:\n\n{context}", NumberSchema)
print(result)

In [ ]:
def ask_ai(field: str, context: str) -> List[Substance] | str:   
    prompts = {
        "title" : {"prompt":f"J'ai besoin d'un titre qui résume en une petite phrase cette description:\n{context}", "schema": TextSchema},
        "fatalities" : {"prompt":f"Combien de morts y a t il dans la description suivante:\n{context}\n\n\nNe répond qu'un seul nombre", "schema": NumberSchema},
        "injuries" : {"prompt":f"Combien de blessés y a t il dans la description suivante:\n{context}\n\n\nNe répond qu'un seul nombre", "schema": NumberSchema},
        "evacuated" : {"prompt":f"Combien de personnes évacuées y a t il dans la description suivante:\n{context}\n\n\nNe répond qu'un seul nombre", "schema": NumberSchema},
        "hospitalized" : {"prompt":f"Combien de personnes hospitalisées y a t il dans la description suivante:\n{context}\n\n\nNe répond qu'un seul nombre", "schema": NumberSchema},
        "substances" : {"prompt":f"Quelles substances sont en jeu dans la description suivante:\n{context}\n\n\nS'il n'y en a pas répond un JSON vide. Si la quantité n'est pas renseignée, ne met rien", "schema":SubstancesOutput},
    }

    if field not in prompts.keys():
        return "<AI> To Prompt"

    selectedPrompt = prompts[field]["prompt"]
    selectedSchema = prompts[field]["schema"]
    response = cache.extract(selectedPrompt, selectedSchema).response
    
    print(field, response)
    return response

In [ ]:
def convert_to_db(df : pd.DataFrame, trunc = None):    

    if trunc is not None :
        df = df.head(trunc)

    def create_line(line : pd.Series):
        address = "NULL"
        site_id = str(uuid5(UUID_NAMESPACE, address))
        sites = {
            "site_id" : site_id,
            "plant_name": "",
            "address": address,
            "latitude": None,
            "longitude": None,
            "country": "France",
            "industrial_activity": line["Comité technique national"].split(' - ')[1],
        }

        CONTEXT_FOR_AI = line["Résumé de l'accident"]
        title : str = ask_ai("title", CONTEXT_FOR_AI) # type: ignore

        accident_key = " ".join([title, str(line["Numéro du dossier"])])
        accident_id = str(uuid5(UUID_NAMESPACE, accident_key))
        accidents = {
            "accident_id": accident_id,
            "site_id": site_id,
            "title": title,
            "source": "EPICEA",
            "source_id": str(line["Numéro du dossier"]),
            "accident_date": "NULL",
            "severity_scale": "NULL",
            "raw_data": "", #line,
            "created_at": "date.now()",
            "updated_at": "",
        }

        causes = {
            "accident_id": accident_id,
            "event_category": line["Code entreprise"].split(' - ')[1],
            "equipment_failure": line["Matériel en cause"],
            "description": line["Résumé de l'accident"], 
        }

        substancesOutput : List[Substance] = ask_ai("substances", CONTEXT_FOR_AI) # type: ignore
        print("============================", substancesOutput)
        substancesArray = []
        for substance in substancesOutput :
            substanceJSON = {
                "name":substance.name,
                "cas_number":substance.cas_number,
                "quantity":substance.quantity,
                "clp_class":substance.clp_class
            }
            substancesArray.append(substanceJSON)
        substances = {"substances":substancesArray}

        consequences_human = {
            "accident_id": accident_id,
            "fatalities": ask_ai("fatalities", CONTEXT_FOR_AI),
            "injuries": ask_ai("injuries", CONTEXT_FOR_AI),
            "evacuated": ask_ai("evacuated", CONTEXT_FOR_AI),
            "hospitalized": ask_ai("hospitalized", CONTEXT_FOR_AI),
        }

        consequences_other = {
            "accident_id": accident_id,
            "environmental_impact": ask_ai("environmental_impact", CONTEXT_FOR_AI),
            "economic_cost": ask_ai("economic_cost", CONTEXT_FOR_AI),
            "disruption_duration": ask_ai("disruption_duration", CONTEXT_FOR_AI)
        }

        tables = {
            "sites": sites,
            "accidents": accidents,
            "causes": causes,
            "substances": substances,
            "consequences_human": consequences_human,
            "consequences_other": consequences_other
        }
        
        return tables
    
    db_lines = []

    for x in tqdm(iter(df.iloc), total=trunc, ncols=200):
        db_lines.append(create_line(x))

    return db_lines

df = pd.DataFrame(epicea_example, index=[0])
EPICEA_db_jsons = convert_to_db(df)
print(type(EPICEA_db_jsons))
print_db_jsons(EPICEA_db_jsons)

# Database Inserting

In [ ]:
from psycopg2.extras import execute_values

def insert_jsons_in_db(db_jsons, conn):
    cur = conn.cursor()

    # 1. Insert sites
    # Generate an array of tuples without duplicates
    constraint_set = set()
    sites_tuples = []
    for db_json in db_jsons:
        constraint_key = db_json["sites"]["plant_name"] + db_json["sites"]["address"]
        if constraint_key in constraint_set : continue
        constraint_set.add(constraint_key)

        sites_tuples.append((
            db_json["sites"]["site_id"],
            db_json["sites"]["plant_name"], 
            db_json["sites"]["address"], 
            db_json["sites"]["latitude"], 
            db_json["sites"]["longitude"], 
            db_json["sites"]["country"], 
            db_json["sites"]["industrial_activity"]
        ))

    print("Inserting sites")
    execute_values(cur, """INSERT INTO sites (site_id, plant_name, address, latitude, longitude, country, industrial_activity) VALUES %s ON CONFLICT (plant_name, address) DO NOTHING""", sites_tuples)

    cur.execute("""
        SELECT site_id, plant_name, address 
        FROM sites
    """)
    all_sites = cur.fetchall()
    site_mapping = {(row[1], row[2]): row[0] for row in all_sites}

    # Insert accidents  
    accidents_tuples = [
        (
            db_json["accidents"]["accident_id"], 
            site_mapping[(db_json["sites"]["plant_name"], db_json["sites"]["address"])], 
            db_json["accidents"]["title"], 
            db_json["accidents"]["source"], 
            db_json["accidents"]["source_id"], 
            db_json["accidents"]["accident_date"], 
            db_json["accidents"]["severity_scale"]
        ) 
        for db_json in db_jsons
    ]
    print("Inserting accidents")
    execute_values(cur, """INSERT INTO accidents (accident_id, site_id, title, source, source_id, accident_date, severity_scale) VALUES %s ON CONFLICT DO NOTHING""", accidents_tuples)

    # Insert causes
    causes_tuples = [
        (
            db_json["causes"]["accident_id"], 
            db_json["causes"]["event_category"], 
            db_json["causes"]["equipment_failure"], 
            db_json["causes"]["description"]
        ) 
        for db_json in db_jsons
    ]
    print("Inserting causes")
    execute_values(cur, """INSERT INTO causes (accident_id, event_category, equipment_failure, description) VALUES %s ON CONFLICT (accident_id) DO NOTHING""", causes_tuples)

    # Insert substances
    substances_tuples = [
        (
            db_json["substances"]["accident_id"], 
            db_json["substances"]["name"], 
            db_json["substances"]["cas_number"], 
            db_json["substances"]["quantity"], 
            db_json["substances"]["clp_class"]
        ) 
        for db_json in db_jsons
    ]
    print("Inserting substances")
    execute_values(cur, """INSERT INTO substances (accident_id, name, cas_number, quantity, clp_class) VALUES %s ON CONFLICT (accident_id) DO NOTHING""", substances_tuples)

    # Insert human consequences
    human_tuples = [
        (
            db_json["consequences_human"]["accident_id"], 
            db_json["consequences_human"]["fatalities"], 
            db_json["consequences_human"]["injuries"], 
            db_json["consequences_human"]["evacuated"], 
            db_json["consequences_human"]["hospitalized"]
        ) 
        for db_json in db_jsons
    ]
    print("Inserting consequences_human")
    execute_values(cur, """INSERT INTO consequences_human (accident_id, fatalities, injuries, evacuated, hospitalized) VALUES %s ON CONFLICT (accident_id) DO NOTHING""", human_tuples)

    # Insert other consequences
    other_tuples = [
        (
            db_json["consequences_other"]["accident_id"], 
            db_json["consequences_other"]["environmental_impact"], 
            db_json["consequences_other"]["economic_cost"], 
            db_json["consequences_other"]["disruption_duration"]
        ) 
        for db_json in db_jsons
    ]
    print("Inserting consequences_other")
    execute_values(cur, """INSERT INTO consequences_other (accident_id, environmental_impact, economic_cost, disruption_duration) VALUES %s ON CONFLICT (accident_id) DO NOTHING""", other_tuples)

    # Commit all inserts
    conn.commit()
    cur.close()

conn_string = os.getenv("NEON_CONNECTION_STRING")

with psycopg2.connect(conn_string) as conn : 
    insert_jsons_in_db(ARIA_db_jsons, conn)